In [ ]:
import json
import requests
from datetime import datetime
from collections import defaultdict
from time import sleep

#https://pypi.org/project/gamma-viewer/
# from gamma_viewer import GammaViewer
from IPython.display import display

In [ ]:
def log_query(query, ars_url, pk, time=datetime.now()):
    try:
        message = f'''
{json.dumps(query, indent=4)}
{ars_url}
{pk}
{time}
'''    
        with open('workflow_b_ars_log.txt', 'a') as fh:
            fh.write(message)
    except:
        print('Could not write to log file')
        
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    
    t = datetime.now()
    print(f'Submitted at {t} to {ars_url}')
    log_query(m, ars_url, message_id, t)
    
    return message_id

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

def submit_to_devars(m):
    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_devars_results(m):
    return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

def print_json(j):
    printjson(j)
    
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())
    
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

def send_trapi(m, service_name=None, service_url=None):
    services = {
        'rtx-kg2': 'https://arax.ncats.io/api/rtxkg2/v1.2/query',
        'icees-asthma': 'https://icees.renci.org:16339/query?reasoner=true&verbose=false',
        'icees-dili': 'https://icees.renci.org:16341/query?reasoner=true&verbose=false',
        'arax': 'https://arax.ncats.io/api/arax/v1.2/query'
    }
    
    if service_url is None and service_name is not None:
        service_url = services.get(service_name.lower())
        
    if service_url is None:
        print('No service URL')
       
    t1 = datetime.now()
    print(f'Submitting at {t1} to {service_url}')
    response = requests.post(service_url,json=m)
    try:
        j = response.json()
    except:
        print('fail')
        j = None    
    
    t2 = datetime.now()    
    print(f'Query took {(t2-t1).total_seconds()} seconds')
    
    return j

def rtx_result_summary(response):
    print(response['description'])
    n_results = len(response['message']['results'])
    print(f"Number of results: {n_results}")
    if n_results:
        print(f"Number of nodes bound to n2: {len(response['message']['results'][0]['node_bindings']['n2'])}")
        print(f"Number of nodes bound to n3: {len(response['message']['results'][0]['node_bindings']['n3'])}")
        
def look_for_desired_answers(response):
    looking_for=['resveratrol', 'prednisone', 'dexamethasone', 'silybin']
    for x in response['message']['results'][0]['node_bindings']['n3']:
        name = response['message']['knowledge_graph']['nodes'][x['id']]['name']
        for desired in looking_for:
            if desired in name:
                print(name)

# Workflow B: DILI Drug Repurposing

**Background**

DILI (drug-induced liver injury) is the outcome of a drug exposure in rare patients with heightened sensitivity to drug-related hepatic sequela. Drugs account for 20-40% of all instances of fulminant hepatic failure, and drug-induced hepatic injury is the most common cited reason for withdrawal of an approved drug from the market.

While the etiology of DILI is not completely understood, it is believed to be the result of an immune response gone awry.

DILI has only one consensus treatment: discontinuation of the causal agent. While DILI may resolve after the causal agent is discontinued, complete resolution may require months or years and, in many cases, the condition deterioriates before resolving and/or becomes chronic. Moreover, DILI is associated with high healthcare costs and high morbidity and mortality, often leading to hospitalization, liver transplant, and other poor outcomes.

**Aim**

The intent of Workflow B is to suggest drugs for repurposing in the treatment of DILI, along with evidence of biological plausibility to justify the suggestion. The goal here is to suggest candidates for clinical trials. Indeed, this is of great interest to the international DILI Network Steering Committee, which is arguably the leading authority on DILI and the only body with a continuous 20-year registry of patients with DILI.

* In natural language, one might ask *are there drugs that can be repurposed to treat DILI-associated phenotypes or outcomes, or allow a patient to continue on a drug that is otherwise effective?*

**Broader Impact**

In addition to identifying plausible drugs for testing in DILIN clinical trials, the workflow and demo are intended to: 
1. Showcase the Translator clinical KPs, all of which provide open access to clinical data and/or computational models applied to those data.
2. Provide real-world evidence that might assist with ranking/sorting of answers and/or SME evaluation of results from the third hop.
3. Highlight Translator's technical capabilities, in terms of (a) three-hop queries, (b) batch queries, (c) asynchronous queries, and (d) open clinical data.
* *Note that competitor commercial systems such as Causaly do not support these capabilities, thus setting the open-source Translator system apart from similar knowledge graph–based biomedical question-answering systems.*

**Generalizability**

While this workflow narrative focuses on DILI, the workflow itself is intended to be disease-agnostic. The focus on DILI is only to highlight Translator capabilities in the context of a compelling use case and engaged user community.

**Structure**

The workflow is structured as an initial three-hop TRAPI query that runs from input DiseaseOrPhenotypicFeature CURIE (e.g., DILI `MONDO:0005359` and Toxic liver disease `SNOMEDCT:197354009`) to DiseaseOrPhenotypicFeature to Gene to Drug (including approved drugs). A fourth hop then seeks Translator evidence (clinical and non-clinical KPs) on whether any of the suggested approved drugs are used to treat or are associated with (biolink:treats, biolink:has_real_world_evidence_of_association_with) the diseases or phenotypic features identified from the first hop or other potentially related traits/diseases. The fourth hop also seeks information on whether the candidates are in clinical trials for DILI or any other disease and/or whether they are associated with adverse events. 

**Implementation**

For implementation, the first three-hop TRAPI query will be precomputed, results will be reviewed with Paul Watkins (SME and member of the DILI Network Steering Committee), and one or two drugs will be selected to highlight in the demo. The fourth hop will then be run live at the December demo. 

**Input Curies**

Suggested input CURIES for the initial B.1 three-hop TRAPI queries include:

* DILI - MONDO:0005359
* Toxic liver disease - SNOMEDCT:197354009

[Node Normalizer equivalent identifiers](https://nodenormalization-sri.renci.org/1.2/get_normalized_nodes?curie=MONDO%3A0005359&curie=SNOMEDCT%3A197354009&conflate=true)

**Reference Information**

For reference, here is the [workflow mural board](https://app.mural.co/t/ncats3030/m/ncats3030/1620608471364/d9d6ca5aefb8c7af4f756312d2500f0a3f465008), [pre-relay meeting materials](https://drive.google.com/drive/folders/1sCA6iouNHOh9I4ivXrR6DCct6fGgXbXp?usp=sharing), and [Spring 2021 relay meeting materials](https://github.com/ranking-agent/robogallery/tree/master/relay_spring_2021/DILI). Also see mini-hackathon issues #43, #44, #45, #46, #48, and #49.*


# B.1 Overview

The initial three-hop TRAPI query:
1. Starts with DILI and Toxic Liver Disease (henceforth referred to simply as DILI, as these are synonymous terms) and finds diseases and phenotypic features that are observed in real-world evidence (clinical data KPs) to be associated with DILI. 
2. From these associated diseases and phenotypic features, we find genes associated with these conditions in the second hop. 
3. In the third hop, we query Translator for drugs that affect these genes.

In [ ]:
submit_to_devars(json.loads("""
{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                     "ids": ["MONDO:0005359", "SNOMEDCT:197354009"],
                     "categories": ["biolink:DiseaseOrPhenotypicFeature"]
                },
                "n1": {
                    "categories": ["biolink:DiseaseOrPhenotypicFeature"]
                },
                "n2": {
                    "categories": ["biolink:Gene"]
                },
                "n3": {
                    "categories": ["biolink:Drug"]
                }
            },
            "edges": {
                "e01": {
                    "subject": "n0",
                    "object": "n1",
                    "predicates": ["biolink:has_real_world_evidence_of_association_with"]
                },
                "e02": {
                    "subject": "n2",
                    "object": "n1",
                    "predicates": ["biolink:gene_associated_with_condition"]
                },
                "e03": {
                    "subject": "n3",
                    "object": "n2",
                    "predicates": ["biolink:affects"]
                }
            }
        }
    }
}
"""))

**Example answer set**

https://arax.ncats.io/?r=c704317a-2987-452b-bd9e-134458f05190  

ARAX results find:
* **#6 quercetin:** 31/500 ARAX responses identify quercetin, a dietary flavonoid with antioxidant and anti-inflammatory properties
* **#12 dexamethasone:** 37/500 ARAX responses identify dexamethasone, a glucocorticoid and regulated drug.
* **#85 resveratrol:**  40/500 ARAX responses identify resveratrol, an unregulated antioxidant and component of red grape skin.


ARAX results, 07.07.2022: PK = f05aa00e-26e7-45d7-91e0-313db66a52d1, PK = 8a879134-7353-4f3d-95ef-8512488e24fb (increased timeout) 
Of note, there is anecdotal evidence and a few publications to support the use of these pharmaceuticals or nutraceuticals in the treatment of DILI.

# B.2 Overview

A fourth hop then seeks Translator evidence (clinical and non-clinical KPs) on whether any of the suggested candidates are used to treat (biolink:treats, biolink:has_real_world_evidence_of_association_with) the diseases or phenotypic features identified from the first hop, or other potentially related traits/diseases, and whether they are associated with adverse events (i.e., an indicator of the safety profile). 

A few example candidate drugs of interest that can be used according to user interest include:
* Dexamethasone - `CHEBI:41879`
* Resveratrol - `MESH:D000077185`
* Silybin - `MESH:D000077385`
* Methylprednisolone - `["PUBCHEM.COMPOUND:5877", "PUBCHEM.COMPOUND:124653", "PUBCHEM.COMPOUND:6741"]`
* Prednisone - `PUBCHEM.COMPOUND:5865`
* Quercetin - `CHEMBL.COMPOUND:CHEMBL50`

In [ ]:
submit_to_ars(json.loads(
"""
{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "ids": ["MESH:D000077185"],
                    "categories": [
                        "biolink:ChemicalEntity"
                    ],
                    "name": "Resveratrol"
                },
                "n1": {
                    "categories": [
                        "biolink:DiseaseOrPhenotypicFeature"
                    ]
                }
            },
            "edges": {
                "e0": {
                    "subject": "n0",
                    "object": "n1",
                    "predicates": ["biolink:treats", "biolink:has_real_world_evidence_of_association_with"]
                }
            }
        }
    }
}
"""
))

**Example answer set**

https://arax.ncats.io/?r=c0213b7d-9634-464a-8c9e-5d210b602771

# Specific narrative

# B.1 three-hop query

PK = 70ec1775-8b71-4442-8a26-d16110d3351b, tracker 2021-10-20

A quick review of the query results finds answer contributions from a number of Translator KPs as contributed by one ARA using a new Translator asynchronous query option.
 
* e01: Biothings Explorer Provider, Clinical Data Provider, Clinical Risk Provider, Exposures Provider
* e02: Genetics Provider, RTX-KG2 Provider, Tumor Gene Mutation Provider
* e03: RTX-KG2 Provider
 
Several answers appear interesting, including [resveratrol](https://en.wikipedia.org/wiki/Resveratrol) (#27 of top 500) and [quercetin](https://en.wikipedia.org/wiki/Quercetin) (#401 of top 500).

# Part I. Resveratrol (Answer #27)

**B.1 three-hop query**

When we examine the first edge, `biolink:has_real_world_evidence_of_association_with`, in the query results for answer #27, we see answer contributions from two clinical KPs that expose real-world evidence, both indicating an association between DILI and autoimmune hepatitis.
 
Exposures Provider provides data on a unique real-world longitudinal cohort of patients with expert-confirmed DILI, as opposed to EHR assertions based on medical coding data and without a definitve diagnostic code for DILI. 

* The Exposures Provider DILI cohort was developed and is maintained by the international DILI Network — arguably the leading worldwide authority on DILI.

* Exposures Provider finds that 1 of 265 patients (0.4%) with expert-confirmed DILI also have autoimmune hepatitis.

```
    {
      "frequency": 264,
      "percentage": 0.9962264150943396
    },
    {
      "frequency": 1,
      "percentage": 0.0037735849056603774
    }
  ],
  "total": 265
```




Clinical Risk Provider exposes logistic regression estimates derived from EHR data on a cohort of roughly 20M patients within Providence/Swedish Health Services hospital system.

* Clinical Risk Provider finds an increased risk of autoimmune hepatitis given a diagnosis of toxic liver disease (synonymous with DILI); the following evidence is provided to support the model prediction.

```
attribute_type_id:
auc_roc
value:
0.914

attribute_type_id:
classifier
value:
Logistic Regression

attribute_type_id:
feature_coefficient
value:
3.1809
```




The [STAT3 gene](https://www.genecards.org/cgi-bin/carddisp.pl?gene=STAT3) is suggested as the intermediary gene between autoimmune hepatitis and resveratrol. This gene encodes a protein product of the STAT family. STAT3 is activated by a variety of cytokines and plays a role in inflammation, apoptosis, host response, and a variety of other processes.

> Clicking on the `STAT3 node` takes you to a number of hyperlinks, which provide additional information on the gene.
 
> Clicking on the `resveratrol node`, we find additional information on the compound, as well as hyperlinks to outside sources of information.
 
> Clicking on the edge `biolink:has_normalized_google_distance_with `provides publication support for the association between STAT3 and resveratrol.

* The [first publication](https://www.ncbi.nlm.nih.gov/pubmed/22822401) describes the ‘French paradox’ and the benefits of resveratrol as “broadly studied and shown to have antioxidant, anti-inflammatory, anti-proliferative, and anti-angiogenic effects, with those on oxidative stress possibly being most important and underlying some of the others, but many signaling pathways are among the molecular targets of resveratrol”.

**B.2 one-hop query - resveratrol**
 
PK = 978f0101-4eae-44e6-8033-182d81c79d08, tracker 10.20.2021
 
When we then ask Translator for other evidence related to resveratrol, including its use in the real-world as well as the evidence derived from curated knowledge sources.

The top 50 results from Clinical Data Provider — a clinical KP that exposes EHR data on a five-year cohort of roughly 2M patients from Columbia University Irving Medical Center's OHDSI dataset — show that resveratrol is associated with diagnoses for a variety of diseases, including the inflammatory conditions psoriasis (#31) and osteoarthritis (#33).

* Psoriasis

| attribute_source | attribute_type_id | description | original_attribute_name | value_type_id | value | 
| :-- | :-- | :-- | :-- | :-- | :-- | 
| infores:cohd | biolink:has_evidence | Observed-expected frequency ratio. http://cohd.io/about.html | ln_ratio | EDAM:data_1772  | 2.46 |
| infores:cohd | biolink:has_confidence_level | 	Observed-expected frequency ratio 0.99% confidence interval | ln_ratio_confidence_interval | EDAM:data_0951  | [1.30, 3.12] |

```
attribute_source: infores:cohd
attribute_type_id: biolink:has_evidence
description: Observed-expected frequency ratio. http://cohd.io/about.html
original_attribute_name: ln_ratio
value_type_id: EDAM:data_1772
value: 2.46
```
* Osteoarthritis

| attribute_source | attribute_type_id | description | original_attribute_name | value_type_id | value | 
| :-- | :-- | :-- | :-- | :-- | :-- | 
| infores:cohd | biolink:has_evidence | Observed-expected frequency ratio. http://cohd.io/about.html | ln_ratio | EDAM:data_1772  | 2.47 |
| infores:cohd | biolink:has_confidence_level | 	Observed-expected frequency ratio 0.99% confidence interval | ln_ratio_confidence_interval | EDAM:data_0951  | [1.24, 3.10] |

```
attribute_source: infores:cohd
attribute_type_id: biolink:has_evidence
description: Observed-expected frequency ratio. http://cohd.io/about.html
original_attribute_name: ln_ratio
value_type_id: EDAM:data_1772
value: 2.47
```






All six ARAs return responses to the query:

* Ranking Agent returns 1847 answers.

* Improving Agent indicates that resveratrol is used to treat fatty liver disease (#13 of 16 answers).

* Unsecret Agent indicates that resveratrol is used to treat fatty liver (#78 of 479 answers), fatty liver disease (#79), cirrhosis (#135), liver diseases (#136), alcoholic liver disease (#137), liver damage (#139), liver injury (#247), liver fibrosis (#265), liver cancer (#289, #322, #387), chronic liver disease (#306), NAFLD (#316), liver dysfunction (#401), and hepatotoxicity (#269).

* Explanatory Agent identifies 20 of 504 answers suggesting that resveratrol treats a variety of liver-related conditions.

* Exploring Agent returns 708 answers, including #669 indicating that resveratrol is used to treat drug-induced liver injury.

* Expander Agent likewise finds evidence that resveratrol treats drug-induced liver injury (#169/500).

> Clicking on the `biolink:has_normalized_google_distance_with` finds three publications supporting a protective effect of resveratrol in hepatotoxicity induced by a variety of chemical agents in mouse models.

> Expander Agent finds additional evidence that resveratrol treats drug-related side effects and adverse reactions (#46) and drug rash with eosinophilia and systemic symptoms (#444).




**SME Evaluation**

Our SME, Dr. Paul Watkins, notes that resveratrol has limited published support for its effectiveness in the prevention of DILI.

* Dr. Watkins considers this candidate as viable for rigorous evaluation as part of a DILI Network clinical trial.


# Part II. Quercetin - Answer #401

**B.1 three-hop query**

When we examine the first edge in the query results, we see answer contributions from two clinical KPs that expose real-world evidence. 

However, unlike the results for resveratrol, the results for quercetin include answer contributions from Clinical Data Provider and Clinical Risk Provider, with both clinical KPs supporting an association between toxic liver disease (equivalent to DILI) and infectious bacterial disease with sepsis.

* Clicking on the edge for 
`COHD:SNOMEDCT:197354009-biolink:has_real_world_evidence_of_association_with-MONDO:0005229` shows the following support for this assertion, including:

| attribute_source | attribute_type_id | description | original_attribute_name | value_type_id | value | 
| :-- | :-- | :-- | :-- | :-- | :-- | 
| infores:cohd | biolink:has_evidence | Observed-expected frequency ratio. http://cohd.io/about.html | ln_ratio | EDAM:data_1772  | 4.48 |
| infores:cohd | biolink:has_confidence_level | 	Observed-expected frequency ratio 0.99% confidence interval | ln_ratio_confidence_interval | EDAM:data_0951  | [	3.63, 5.00] |

```
attribute_source:infores: cohd
attribute_type_id: biolink:has_evidence
description: Observed-expected frequency ratio. http://cohd.io/about.html
original_attribute_name: ln_ratio
value_type_id: EDAM:data_1772
value: 4.48

```





Clicking on the edge for `ClinicalRiskKP:UMLS:C0348754-biolink:has_real_world_evidence_of_association_with-HP:0100806` provides the following support:
```
attribute_type_id:
auc_roc
value:
0.9452

attribute_type_id:
classifier
value:
Logistic Regression

attribute_type_id:
feature_coefficient
value:
0.9674

```




[TNF](https://www.genecards.org/cgi-bin/carddisp.pl?gene=TNF&keywords=tnf) (tumor necrosis factor) is identified as the gene that mediates the relationship between infectious bacterial disease with sepsis and quercetin.
TNF is a proinflammatory cytokine that is involved with multiple immune processes.
 
* Clicking on the link for `biolink:has_normalized_google_distance_with` reveals more than two dozen publications supporting a relationship between TNF and quercetin, with most publications derived from primary rodent studies.

* The [first publication](https://pubmed.ncbi.nlm.nih.gov/10454017/) is titled: “Quercetin inhibits LPS-induced nitric oxide and tumor necrosis factor-alpha production in murine macrophages”, suggesting that quercetin inhibits TNF.

**B.2 one-hop query - quercetin**

PK =  3a50ba27-652e-4832-baec-6f5c83793944, tracker 10.25.2021
 
When we then ask Translator what else it knows about quercetin, we find responses from both clinical KPs and non-clinical curated KPs.

Clinical Data Provider identifies 10 associations between quercetin and a variety of diseases, including allergic rhinitis.

* Allergic rhinitis (#4)

| attribute_source | attribute_type_id | description | original_attribute_name | value_type_id | value | 
| :-- | :-- | :-- | :-- | :-- | :-- | 
| infores:cohd | biolink:has_evidence | Observed-expected frequency ratio. http://cohd.io/about.html | ln_ratio | EDAM:data_1772  | 2.24 |
| infores:cohd | biolink:has_confidence_level | 	Observed-expected frequency ratio 0.99% confidence interval | ln_ratio_confidence_interval | EDAM:data_0951  | [1.30, 2.79] 

```
attribute_source:	infores:cohd
attribute_type_id:	biolink:has_evidence
description:	Observed-expected frequency ratio. http://cohd.io/about.html
original_attribute_name:	ln_ratio
value_type_id:	EDAM:data_1772
value:	2.24
```

Five of six ARAs also return responses to the query.

* ARAGORN returns 1697 answers.

* Explanatory Agent returns 2996 answers.

* BTE returns 473 answers, with many contributed by Text Mining Provider.

* ARAX identifies quercetin as a treatment for drug-induced liver injury (#106 of top 500 answers), drug-induced dyskinesia (#257), and drug-related side effects and adverse reactions (#372), with support drawn from rodent studies.

> Clicking on `biolink:has_normalized_google_distance_of` identifies more than two dozen supporting publications for the evidence supporting the use of quercetin in drug-induced liver injury, largely drawn from primary rodent studies.

> The [first publication](https://pubmed.ncbi.nlm.nih.gov/32253952/) is titled “Involvement of P450s and nuclear receptors in the hepatoprotective effect of quercetin on liver injury by bacterial lipopolysaccharide”.

* Unsecret Agent returns 11 answers indicating that quercetin is used to treat a variety of conditions.

> Clicking on the first answer, drawn from ChemBL, shows that [quercetin is in clinical trials](https://clinicaltrials.gov/search?id=%22NCT01708278%22OR%22NCT03989271%22) as a treatment for chronic obstructive pulmonary disorder, thus demonstrating precedence for clinical trials on quercetin.

**SME Evaluation**

Our SME, Dr. Watkins, notes that quercetin is an antioxidant that is generally regarded as safe.

* He considers quercetin as a second viable candidate for rigorous evaluation as part of a DILI Network clinical trial.
